In [1]:
import tweepy
import pandas as pd
import nltk
import string
import os
import re
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import udf
from pyspark.ml.feature import CountVectorizer
consumer_key = "Ax79DI6gXdMjOCQyQ61v2Khbj"
consumer_secret = "DBSmfUL50FlhpVnePa373lOpBFffJjrGInXbSGG4T7Ze2RRQTl"
access_token = "1418173405988376579-9g3SF5klnWOgd62nUfTfaqrQdzkJRA"
access_token_secret = "q3jO28xbGDlvE6djCNeoSMQbtkosZYT7mBTv0O6wznMpw"
# Creating the authentication object
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
# Setting your access token and secret
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)


In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

spark = SparkSession.builder.master('local[*]').appName('classifier').getOrCreate()
df = spark.read.csv("trainingandtestdata/training.1600000.processed.noemoticon.csv")
df=df.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df2=spark.read.csv("trainingandtestdata/testdata.manual.2009.06.14.csv")
df2=df2.toDF('polarity','tweet_id','tweet_date','query_string','tweet_screen_name','Tweet')
df.show(5)
df2.show(5)

+--------+----------+--------------------+------------+-----------------+--------------------+
|polarity|  tweet_id|          tweet_date|query_string|tweet_screen_name|               Tweet|
+--------+----------+--------------------+------------+-----------------+--------------------+
|       0|1467810369|Mon Apr 06 22:19:...|    NO_QUERY|  _TheSpecialOne_|@switchfoot http:...|
|       0|1467810672|Mon Apr 06 22:19:...|    NO_QUERY|    scotthamilton|is upset that he ...|
|       0|1467810917|Mon Apr 06 22:19:...|    NO_QUERY|         mattycus|@Kenichan I dived...|
|       0|1467811184|Mon Apr 06 22:19:...|    NO_QUERY|          ElleCTF|my whole body fee...|
|       0|1467811193|Mon Apr 06 22:19:...|    NO_QUERY|           Karoli|@nationwideclass ...|
+--------+----------+--------------------+------------+-----------------+--------------------+
only showing top 5 rows

+--------+--------+--------------------+------------+-----------------+--------------------+
|polarity|tweet_id|        

In [3]:
df=df[['tweet_id','query_string','Tweet','polarity']]
df2=df2[['tweet_id','query_string','Tweet','polarity']]
df.groupBy('polarity').count().orderBy('count',ascending=False).show(10,False)
df2.groupBy('polarity').count().orderBy('count',ascending=False).show(10,False)
#df=df.toPandas()
#df.head(5)

+--------+------+
|polarity|count |
+--------+------+
|0       |800000|
|4       |800000|
+--------+------+

+--------+-----+
|polarity|count|
+--------+-----+
|4       |182  |
|0       |177  |
|2       |139  |
+--------+-----+



In [4]:
from nltk.corpus.reader.wordnet import *
from sklearn.feature_extraction.text import TfidfTransformer
from nltk.tokenize import word_tokenize
from pyspark.sql.types import StringType,DoubleType,IntegerType
wn = nltk.WordNetLemmatizer()
worddict = set(nltk.corpus.words.words())

def preprocessing(text):
    wordset_n = set(wn.lemmatize(w, NOUN) for w in word_tokenize(text.lower().strip()))
    wordset_v = set(wn.lemmatize(w, VERB) for w in wordset_n)
    wordset = set(wn.lemmatize(w, ADJ) for w in wordset_v)
    wordset = wordset & worddict
    return ' '.join(list(wordset))


brand_udf=udf(preprocessing,StringType())
df=df.withColumn('text',brand_udf(df['Tweet']))
df=df.withColumn('clean_len',F.length('text'))
df2=df2.withColumn('text',brand_udf(df2['Tweet']))
df2=df2.withColumn('clean_len',F.length('text'))
#tem=df.union(df2)
#tem.show()

In [24]:
print(brand_udf)

<function preprocessing at 0x7fe932e951f0>


In [5]:
from pyspark.ml.feature import  Tokenizer


tokenizer = Tokenizer(inputCol="text", outputCol="words")
#tem = tokenizer.transform(tem)
df=tokenizer.transform(df)
df2=tokenizer.transform(df2)

In [6]:
from pyspark.ml.feature import CountVectorizer
count = CountVectorizer (inputCol="words", outputCol="rawFeatures")
#model = count.fit(tem)
#tem = model.transform(tem)
#tem.show()
model1=count.fit(df)
df=model1.transform(df)


In [7]:
model2=count.fit(df2)
df2=model2.transform(df2)

In [8]:
from pyspark.ml.feature import  IDF
idf = IDF(inputCol="rawFeatures", outputCol="features")
#idfModel = idf.fit(tem)
#tem = idfModel.transform(tem)
#tem = tem.withColumn('polarity', tem['polarity'].cast(IntegerType()))
idfModel = idf.fit(df)
df = idfModel.transform(df)

df = df.withColumn('polarity', df['polarity'].cast(IntegerType()))
idfModel = idf.fit(df2)


In [9]:
df2 = idfModel.transform(df2)
df2 = df2.withColumn('polarity', df2['polarity'].cast(IntegerType()))
trainingDF,testDF = df.randomSplit([0.7, 0.3])



In [34]:
from pyspark.ml.feature import FeatureHasher


fhash = FeatureHasher(inputCols=["text","words","rawFeatures","features"], outputCol ="features")

In [47]:
trainingDF.select("words").show()

+--------------------+
|               words|
+--------------------+
|[get, day, that, ...|
|[result, cry, sch...|
|[whole, feel, my,...|
|[at, mad, no, all...|
|[the, crew, whole...|
|      [a, hug, need]|
|[fine, only, i, h...|
|                [me]|
|  [i, just, my, ear]|
|[i, talk, to, you...|
|[really, i, would...|
|[miss, get, wa, i...|
|[out, death, will...|
|[tax, about, to, ...|
|[out, drink, you,...|
|[i, a, cake, but,...|
|[this, hop, go, i...|
|[at, class, tomor...|
|[i, when, hate, u...|
|          [now, sad]|
+--------------------+
only showing top 20 rows



In [35]:
featDF = fhash.transform(trainingDF)

IllegalArgumentException: requirement failed: FeatureHasher requires columns to be of numeric, boolean or string. Column words was array<string>

In [10]:
from pyspark.ml.classification import RandomForestClassifier
rf_classifier=RandomForestClassifier(labelCol='polarity', featuresCol="features")

In [15]:
numFolds = 3
evaluator = MulticlassClassificationEvaluator()

pipeline = Pipeline(stages=[rf_classifier])

paramGrid = (ParamGridBuilder().addGrid(rf_classifier.numTrees, [3, 10]).build())

crossval = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=numFolds)

In [17]:
#from pyspark.ml.classification import RandomForestClassifier
#rf_classifier=RandomForestClassifier(labelCol='polarity', featuresCol="features", numTrees=50).fit(trainingDF.select(['text','words','rawFeatures','features','polarity']))

rfModel = crossval.fit(trainingDF)


#rf_predictions=rf_classifier.transform(testDF.select(['text','words','rawFeatures','features','polarity']))


IllegalArgumentException: label does not exist. Available: tweet_id, query_string, Tweet, polarity, text, clean_len, words, rawFeatures, features, CrossValidator_3cef58472263_rand, rawPrediction, probability, prediction

In [ ]:
rf_predictions.show(10)

In [ ]:
# Follow every follower of the authenticated user
timeline = api.home_timeline()
data = pd.DataFrame(data=[tweet.text for tweet in timeline], columns=['Tweet'])